# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

Welcome to Lesson 1.

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

### ConversableAgent class in Autogen

## Setup

In [9]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## Define an AutoGen agent

In [10]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",  # ALWAYS, NEVER, or PROMPT. ALWAYS will always use the human input as the prompt, NEVER will never use the human input as the prompt, and PROMPT will use the human input as the prompt if it is not empty.
)

[autogen.oai.client: 07-12 15:39:23] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [12]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke about pediatric dentistry.", "role": "user"}]
)
print(reply)

Why did the tooth go to the dentist by itself? 

Because it wanted to have a little "filling" of its own!


In [13]:
reply = agent.generate_reply(  # generate_reply function does not set internal state of agent
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Sure, which joke would you like me to repeat for you?


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [14]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

[autogen.oai.client: 07-12 15:42:23] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 07-12 15:42:23] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [15]:
chat_result = joe.initiate_chat(  # joe starts convo and recipient is cathy
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Let's do this! So I tried to organize a hide and seek tournament, but it was a total flop. Good news though, I'm unbeaten at it!

--------------------------------------------------------------------------------
joe (to cathy):

That's impressive, Cathy! I guess you could say you're the "hide-and-seek champion by default."

--------------------------------------------------------------------------------
cathy (to joe):

Haha, exactly, Joe! I'm the undisputed champion of hide-and-seek... whether I like it or not!

--------------------------------------------------------------------------------


## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [16]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'role': 'assistant'},
 {'content': "Hey Joe! Let's do this! So I tried to organize a hide and seek "
             "tournament, but it was a total flop. Good news though, I'm "
             'unbeaten at it!',
  'role': 'user'},
 {'content': "That's impressive, Cathy! I guess you could say you're the "
             '"hide-and-seek champion by default."',
  'role': 'assistant'},
 {'content': "Haha, exactly, Joe! I'm the undisputed champion of "
             'hide-and-seek... whether I like it or not!',
  'role': 'user'}]


In [17]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 86,
                                                             'cost': 0.00024400000000000002,
                                                             'prompt_tokens': 230,
                                                             'total_tokens': 316},
                                      'total_cost': 0.00024400000000000002},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 86,
                                                             'cost': 0.00024400000000000002,
                                                             'prompt_tokens': 230,
                                                             'total_tokens': 316},
                                      'total_cost': 0.00024400000000000002}}


In [18]:
pprint.pprint(chat_result.summary)

("Haha, exactly, Joe! I'm the undisputed champion of hide-and-seek... whether "
 'I like it or not!')


## Get a better summary of the conversation

In [19]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Let's do this! So I tried to organize a hide and seek tournament, but it was a total flop. Good news though, I'm unbeaten at it!

--------------------------------------------------------------------------------
joe (to cathy):

That's impressive, Cathy! I guess you could say you're the "hide-and-seek champion by default."

--------------------------------------------------------------------------------
cathy (to joe):

Haha, exactly, Joe! I'm the undisputed champion of hide-and-seek... whether I like it or not!

--------------------------------------------------------------------------------


In [20]:
pprint.pprint(chat_result.summary)

('Cathy jokes about being unbeaten in hide and seek tournaments, leading to '
 "Joe's humorous comment about being the champion by default.")


## Chat Termination

Chat can be terminated using a termination conditions.

In [21]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],  # boolean function - true/false. If true, conversation ends.
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

[autogen.oai.client: 07-12 15:48:27] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 07-12 15:48:27] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [22]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Why did the scarecrow win an award?

Because he was outstanding in his field! 

Okay, your turn! Give me your best joke!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one, Cathy! Here's my joke: Why couldn't the bicycle find its way home?

Because it lost its bearings!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that's a great one, Joe! I love a good pun. Keep 'em coming!

--------------------------------------------------------------------------------
joe (to cathy):

Glad you enjoyed it, Cathy! Here's another one for you: Why did the math book look sad?

Because it had too many problems!

--------------------------------------------------------------------------------
cathy (to joe):


In [23]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

I apologize for the confusion, but as a language model AI, I don't have the capability to remember past interactions once the conversation ends. If you'd like to hear some more jokes or need assistance with anything else, feel free to ask!

--------------------------------------------------------------------------------
cathy (to joe):

I gotta go.

--------------------------------------------------------------------------------
